In [1]:
import os
import mne
import numpy as np
import pandas as pd
from scipy import interpolate
import matplotlib.pyplot as plt
import scipy
from scipy.signal import resample

In [ ]:
SAMPLE_RATE = 128  # fs
SAMPLE_LEN = 128   # T

## Play with the data

In [2]:
root = 'BrainLat/dataset/EEG data/'

In [3]:
# resample the time series data from original_fs to target_fs
def resample_time_series(data, original_fs, target_fs):
    T, C = data.shape
    new_length = int(T * target_fs / original_fs)
    
    resampled_data = np.zeros((new_length, C))
    for i in range(C):
        resampled_data[:, i] = resample(data[:, i], new_length)
        
    return resampled_data

# split the EEG data into segments of length segment_length, dropping the last segment if it is shorter than segment_length
def split_eeg_segments(data, segment_length=128):
    T, C = data.shape
    num_segments = T // segment_length
    reshaped_data = data[:num_segments * segment_length].reshape(num_segments, segment_length, C)

    return reshaped_data

In [4]:
# Feature
feature_path = 'Processed/BrainLat/Feature'
if not os.path.exists(feature_path):
    os.makedirs(feature_path)

In [5]:
bad_subject_list = []
label_list = []
sub_id = 1
for disease_label in os.listdir(root):
    # Skip non-folder files
    if "BrainLat" in disease_label or "SYNAPSE" in disease_label:
        continue
    print(disease_label)
    disease_label_path = os.path.join(root, disease_label)
    for file_folder in os.listdir(disease_label_path):
        if file_folder == 'AR' or file_folder == 'CL':
            print(file_folder)
            file_folder_path = os.path.join(disease_label_path, file_folder)
            for file in os.listdir(file_folder_path):
                file_path = os.path.join(file_folder_path, file)
                set_file_path = ''
                if 'sub' in file_path or 'suj' in file_path:   # 4_MS has different folder name
                    set_file_path = os.path.join(file_path, 'eeg')
                    if not os.path.exists(set_file_path):  # same subjects in 4_MS has different folder structure
                        set_file_path = file_path
                    # print(set_file_path)
                    for set_file in os.listdir(set_file_path):
                        if set_file.endswith('.set'):
                            set_file_path = os.path.join(set_file_path, set_file)
                            print(set_file_path)
                            try:
                                raw = mne.io.read_raw_eeglab(set_file_path, preload=True)
                                # print(raw.info['ch_names'])
                                standard_channels = ['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 
                                                     'Cz', 'C4', 'T4', 'T5', 'P3', 'Pz', 'P4', 'T6', 'O1', 'O2']
                                data = raw.get_data().T
                                print("Raw data shape ", data.shape)
                                data = resample_time_series(data, 512, SAMPLE_RATE)
                                feature_array = split_eeg_segments(data, SAMPLE_LEN)
                                print("Downsampling and segmented data shape ", feature_array.shape)
                                np.save(feature_path + '/feature_{:03d}.npy'.format(sub_id), feature_array)
                                
                                # first column disease label, second column patient ID(count from 1, only count effective patient)
                                if 'HC' in disease_label:
                                    label_list.append(np.array([0,sub_id]))
                                elif 'AD' in disease_label:
                                    label_list.append(np.array([1,sub_id]))
                                elif 'FTD' in disease_label:
                                    label_list.append(np.array([2,sub_id]))
                                elif 'PD' in disease_label:
                                    label_list.append(np.array([3,sub_id]))
                                elif 'MS' in disease_label:
                                    label_list.append(np.array([4,sub_id]))
                                print(f"Raw subject ID: {file}")
                                print(f"Non-empty subject ID: {sub_id}")
                                sub_id += 1
                                print('\n')
                            except:
                                print('Load EEG data error, skip this subject\n')
                                bad_subject_list.append(set_file_path)
                                continue
                        else:  # .ftd file will be read if .set file call it. no need to read it specifically
                            continue
    print("-------------------------------------\n")

1_AD
AR
BrainLat/dataset/EEG data/1_AD\AR\sub-30001\eeg\s6_sub-30001_rs-HEP_eeg.set
Raw data shape  (159971, 128)
Downsampling and segmented data shape  (312, 128, 128)
Raw subject ID: sub-30001
Non-empty subject ID: 1


BrainLat/dataset/EEG data/1_AD\AR\sub-30002\eeg\s6_sub-30002_rs-HEP_eeg.set
Raw data shape  (165534, 128)
Downsampling and segmented data shape  (323, 128, 128)
Raw subject ID: sub-30002
Non-empty subject ID: 2


BrainLat/dataset/EEG data/1_AD\AR\sub-30004\eeg\s6_sub-30004_rs-HEP_eeg.set
Raw data shape  (145923, 128)
Downsampling and segmented data shape  (285, 128, 128)
Raw subject ID: sub-30004
Non-empty subject ID: 3


BrainLat/dataset/EEG data/1_AD\AR\sub-30008\eeg\s6_sub-30008_rs-HEP_eeg.set
Raw data shape  (160931, 128)
Downsampling and segmented data shape  (314, 128, 128)
Raw subject ID: sub-30008
Non-empty subject ID: 4


BrainLat/dataset/EEG data/1_AD\AR\sub-30009\eeg\s6_sub-30009_rs-HEP_eeg.set
Raw data shape  (207033, 128)
Downsampling and segmented data sh

C:\Users\24700\AppData\Local\Temp\ipykernel_38500\3105216534.py:27: RuntimeWarning: Data file name in EEG.data (s6_sub-10005_rs_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-40005_rs_eeg.fdt).
  raw = mne.io.read_raw_eeglab(set_file_path, preload=True)


Raw data shape  (263168, 128)
Downsampling and segmented data shape  (514, 128, 128)
Raw subject ID: sub-40005
Non-empty subject ID: 55


BrainLat/dataset/EEG data/3_PD\CL\sub-40006\eeg\s40006_CH_PD_reject.set
Load EEG data error, skip this subject

BrainLat/dataset/EEG data/3_PD\CL\sub-40007\eeg\sub-40007_rs_eeg.set
Reading C:\Users\24700\PycharmProjects\DataPreprocessingYihe\BrainLat\BrainLat\dataset\EEG data\3_PD\CL\sub-40007\eeg\sub-40007_rs_eeg.fdt
Reading 0 ... 414719  =      0.000 ...   809.998 secs...


C:\Users\24700\AppData\Local\Temp\ipykernel_38500\3105216534.py:27: RuntimeWarning: Data file name in EEG.data (s6_sub-10007_rs_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (sub-40007_rs_eeg.fdt).
  raw = mne.io.read_raw_eeglab(set_file_path, preload=True)


Raw data shape  (414720, 128)
Downsampling and segmented data shape  (810, 128, 128)
Raw subject ID: sub-40007
Non-empty subject ID: 56


BrainLat/dataset/EEG data/3_PD\CL\sub-40008\eeg\sub-40008_rs_eeg.set
Reading C:\Users\24700\PycharmProjects\DataPreprocessingYihe\BrainLat\BrainLat\dataset\EEG data\3_PD\CL\sub-40008\eeg\sub-40008_rs_eeg.fdt
Reading 0 ... 236031  =      0.000 ...   460.998 secs...


...

Raw data shape  (215552, 128)
Downsampling and segmented data shape  (421, 128, 128)
Raw subject ID: suj_533
Non-empty subject ID: 101


BrainLat/dataset/EEG data/4_MS\AR\suj_534\eeg\suj_534_rs_eeg.set
Reading C:\Users\24700\PycharmProjects\DataPreprocessingYihe\BrainLat\BrainLat\dataset\EEG data\4_MS\AR\suj_534\eeg\suj_534_rs_eeg.fdt
Reading 0 ... 218623  =      0.000 ...   426.998 secs...


C:\Users\24700\AppData\Local\Temp\ipykernel_38500\3105216534.py:27: RuntimeWarning: Data file name in EEG.data (s6_sub-100023_rs_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (suj_534_rs_eeg.fdt).
  raw = mne.io.read_raw_eeglab(set_file_path, preload=True)


Raw data shape  (218624, 128)
Downsampling and segmented data shape  (427, 128, 128)
Raw subject ID: suj_534
Non-empty subject ID: 102


BrainLat/dataset/EEG data/4_MS\AR\suj_535\eeg\suj_535_rs_eeg.set
Reading C:\Users\24700\PycharmProjects\DataPreprocessingYihe\BrainLat\BrainLat\dataset\EEG data\4_MS\AR\suj_535\eeg\suj_535_rs_eeg.fdt
Reading 0 ... 223743  =      0.000 ...   436.998 secs...


C:\Users\24700\AppData\Local\Temp\ipykernel_38500\3105216534.py:27: RuntimeWarning: Data file name in EEG.data (s6_sub-100024_rs_eeg.fdt) is incorrect, the file name must have changed on disk, using the correct file name (suj_535_rs_eeg.fdt).
  raw = mne.io.read_raw_eeglab(set_file_path, preload=True)


Raw data shape  (223744, 128)
Downsampling and segmented data shape  (437, 128, 128)
Raw subject ID: suj_535
Non-empty subject ID: 103


-------------------------------------

5_HC
AR
BrainLat/dataset/EEG data/5_HC\AR\sub-100012\eeg\s6_sub-100012_rs_eeg.set
Reading C:\Users\24700\PycharmProjects\DataPreprocessingYihe\BrainLat\BrainLat\dataset\EEG data\5_HC\AR\sub-100012\eeg\s6_sub-100012_rs_eeg.fdt
Reading 0 ... 176941  =      0.000 ...   345.588 secs...
Raw data shape  (176942, 128)
Downsampling and segmented data shape  (345, 128, 128)
Raw subject ID: sub-100012
Non-empty subject ID: 104


BrainLat/dataset/EEG data/5_HC\AR\sub-100015\eeg\s6_sub-100015_rs_eeg.set
Reading C:\Users\24700\PycharmProjects\DataPreprocessingYihe\BrainLat\BrainLat\dataset\EEG data\5_HC\AR\sub-100015\eeg\s6_sub-100015_rs_eeg.fdt
Reading 0 ... 209992  =      0.000 ...   410.141 secs...
Raw data shape  (209993, 128)
Downsampling and segmented data shape  (410, 128, 128)
Raw subject ID: sub-100015
Non-empty subj

In [6]:
for bad_subject in bad_subject_list:
    print(bad_subject)

BrainLat/dataset/EEG data/3_PD\AR\sub-40001\eeg\s40001_AR_PD_reject.set
BrainLat/dataset/EEG data/3_PD\AR\sub-40006\eeg\s40006_AR_PD_reject.set
BrainLat/dataset/EEG data/3_PD\AR\sub-40008\eeg\s40008_AR_PD_reject.set
BrainLat/dataset/EEG data/3_PD\AR\sub-40010\eeg\s40010_AR_PD_reject.set
BrainLat/dataset/EEG data/3_PD\AR\sub-40011\eeg\s40011_AR_PD_reject.set
BrainLat/dataset/EEG data/3_PD\AR\sub-40012\eeg\s40012_AR_PD_reject.set
BrainLat/dataset/EEG data/3_PD\AR\sub-40013\eeg\s40013_AR_PD_reject.set
BrainLat/dataset/EEG data/3_PD\CL\sub-40001\eeg\s40001_CH_PD_reject.set
BrainLat/dataset/EEG data/3_PD\CL\sub-40004\eeg\s40004_CH_PD_reject.set
BrainLat/dataset/EEG data/3_PD\CL\sub-40006\eeg\s40006_CH_PD_reject.set
BrainLat/dataset/EEG data/3_PD\CL\sub-40009\eeg\s40009_CH_PD_reject.set
BrainLat/dataset/EEG data/3_PD\CL\sub-40011\eeg\s40011_CH_PD_reject.set
BrainLat/dataset/EEG data/3_PD\CL\sub-40015\eeg\s00415_CH_PD_reject.set
BrainLat/dataset/EEG data/5_HC\CL\sub-100013\eeg\s6_sub-100013_r

In [7]:
labels = np.array(label_list)
labels

array([[  1,   1],
       [  1,   2],
       [  1,   3],
       [  1,   4],
       [  1,   5],
       [  1,   6],
       [  1,   7],
       [  1,   8],
       [  1,   9],
       [  1,  10],
       [  1,  11],
       [  1,  12],
       [  1,  13],
       [  1,  14],
       [  1,  15],
       [  1,  16],
       [  1,  17],
       [  1,  18],
       [  1,  19],
       [  1,  20],
       [  1,  21],
       [  1,  22],
       [  1,  23],
       [  1,  24],
       [  1,  25],
       [  1,  26],
       [  1,  27],
       [  1,  28],
       [  1,  29],
       [  1,  30],
       [  1,  31],
       [  1,  32],
       [  1,  33],
       [  1,  34],
       [  1,  35],
       [  2,  36],
       [  2,  37],
       [  2,  38],
       [  2,  39],
       [  2,  40],
       [  2,  41],
       [  2,  42],
       [  2,  43],
       [  2,  44],
       [  2,  45],
       [  2,  46],
       [  2,  47],
       [  2,  48],
       [  2,  49],
       [  2,  50],
       [  2,  51],
       [  2,  52],
       [  2,

In [8]:
label_path = 'Processed/BrainLat/Label'
if not os.path.exists(label_path):
    os.makedirs(label_path)
np.save(label_path + '/label.npy', labels)

In [9]:
np.load('Processed/BrainLat/Label/label.npy')

array([[  1,   1],
       [  1,   2],
       [  1,   3],
       [  1,   4],
       [  1,   5],
       [  1,   6],
       [  1,   7],
       [  1,   8],
       [  1,   9],
       [  1,  10],
       [  1,  11],
       [  1,  12],
       [  1,  13],
       [  1,  14],
       [  1,  15],
       [  1,  16],
       [  1,  17],
       [  1,  18],
       [  1,  19],
       [  1,  20],
       [  1,  21],
       [  1,  22],
       [  1,  23],
       [  1,  24],
       [  1,  25],
       [  1,  26],
       [  1,  27],
       [  1,  28],
       [  1,  29],
       [  1,  30],
       [  1,  31],
       [  1,  32],
       [  1,  33],
       [  1,  34],
       [  1,  35],
       [  2,  36],
       [  2,  37],
       [  2,  38],
       [  2,  39],
       [  2,  40],
       [  2,  41],
       [  2,  42],
       [  2,  43],
       [  2,  44],
       [  2,  45],
       [  2,  46],
       [  2,  47],
       [  2,  48],
       [  2,  49],
       [  2,  50],
       [  2,  51],
       [  2,  52],
       [  2,